In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import expr,substring,col,length,cast, regexp_replace,now,rank,dense_rank,row_number,desc,avg,max,sum,year
from pyspark.sql.types import IntegerType
import re

In [2]:
spark = SparkSession.builder.appName("emp").master("local[3]").getOrCreate()

In [3]:
empSchema = "Employee_ID string,Full_Name string , Job_Title string,  Department string,  Business_Unit string, Gender string,Ethnicity string ,Age int, HireDate date , AnnualSalary string , Bonus string,    Country string,  City string ,Exit_Date date"

In [4]:
employee = spark.read.format("csv")\
                     .option("header","true")\
                     .option("dateFormat","m/d/yyyy")\
                     .schema(empSchema)\
                     .load(r'C:\Users\kkafk\IdeaProjects\pycode\scenario based\EmployeeData_new.csv')

employee.show()

+-----------+----------------+--------------------+-----------+--------------------+------+---------+---+----------+------------+-----+-------------+---------+----------+
|Employee_ID|       Full_Name|           Job_Title| Department|       Business_Unit|Gender|Ethnicity|Age|  HireDate|AnnualSalary|Bonus|      Country|     City| Exit_Date|
+-----------+----------------+--------------------+-----------+--------------------+------+---------+---+----------+------------+-----+-------------+---------+----------+
|     E02002|          Kai Le|   Controls Engineer|Engineering|       Manufacturing|  Male|    Asian| 47|2022-01-05|    $92,368 |  0% |United States| Columbus|      null|
|     E02003|    Robert Patel|             Analyst|      Sales|           Corporate|  Male|    Asian| 58|2013-01-23|    $45,703 |  0% |United States|  Chicago|      null|
|     E02004|      Cameron Lo|Network Administr...|         IT|Research & Develo...|  Male|    Asian| 34|2019-01-24|    $83,576 |  0% |        Ch

In [5]:
emp = employee.select("*")\
        .withColumn("salary", expr('substring(AnnualSalary,2,length(AnnualSalary)-1)'))

emp2 = employee.select("*")\
               .withColumn("salary",substring("AnnualSalary",2,100))

emp3 = employee.select("*")\
                .withColumn("salary", regexp_replace(col("AnnualSalary"),"[$,]",""))\
                .withColumn("salary",col("salary").cast(IntegerType()))\
                .sort("department","salary")


emp3.show(3)

+-----------+----------------+---------+----------+--------------------+------+---------+---+----------+------------+-----+-------------+---------+---------+------+
|Employee_ID|       Full_Name|Job_Title|Department|       Business_Unit|Gender|Ethnicity|Age|  HireDate|AnnualSalary|Bonus|      Country|     City|Exit_Date|salary|
+-----------+----------------+---------+----------+--------------------+------+---------+---+----------+------------+-----+-------------+---------+---------+------+
|     E02096|      Gianna Lim|  Analyst|Accounting|Research & Develo...|Female|    Asian| 46|2006-01-01|    $42,532 |  0% |        China|Chongqing|     null| 42532|
|     E02144|Raelynn Espinoza|  Analyst|Accounting|           Corporate|Female|   Latino| 46|2002-01-12|    $44,411 |  0% |       Brazil|   Manaus|     null| 44411|
|     E02440|     Jaxson Shah|  Analyst|Accounting|           Corporate|  Male|    Asian| 29|2018-01-25|    $44,520 |  0% |United States|  Chicago|     null| 44520|
+---------

In [6]:
mywin1 = Window.partitionBy("department").orderBy(desc("salary"))

df = emp3.withColumn("Rank",rank().over(mywin1))\
         .filter(col("Rank")<=3)
df.printSchema()

root
 |-- Employee_ID: string (nullable = true)
 |-- Full_Name: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Business_Unit: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- HireDate: date (nullable = true)
 |-- AnnualSalary: string (nullable = true)
 |-- Bonus: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Exit_Date: date (nullable = true)
 |-- salary: integer (nullable = true)
 |-- Rank: integer (nullable = false)



In [7]:
winSepc = Window.partitionBy("department").orderBy("salary").rangeBetween(Window.unboundedPreceding,0)

df2 = df.withColumn("cumulative_sum",sum(col("salary")).over(winSepc))
df2.show()

+-----------+--------------------+--------------+---------------+--------------------+------+---------+---+----------+------------+-----+-------------+--------------+----------+------+----+--------------+
|Employee_ID|           Full_Name|     Job_Title|     Department|       Business_Unit|Gender|Ethnicity|Age|  HireDate|AnnualSalary|Bonus|      Country|          City| Exit_Date|salary|Rank|cumulative_sum|
+-----------+--------------------+--------------+---------------+--------------------+------+---------+---+----------+------------+-----+-------------+--------------+----------+------+----+--------------+
|     E02675|          Everett Le|Vice President|     Accounting|           Corporate|  Male|    Asian| 45|2014-01-07|   $250,639 | 34% |        China|      Shanghai|      null|250639|   3|        250639|
|     E02171|           Mia Jiang|Vice President|     Accounting|       Manufacturing|Female|    Asian| 42|2015-01-19|   $255,892 | 38% |United States|         Miami|      null|255

In [8]:
data = [
    (1, "Alice", "Engineering", 70000, "2022-01-15"),
    (1, "Alice", "Engineering", 68000, "2021-01-15"),
    (1, "Alice", "Engineering", 66000, "2020-01-15"),
    (2, "Bob", "HR", 50000, "2022-01-15"),
    (2, "Bob", "HR", 48000, "2021-01-15"),
    (2, "Bob", "HR", 46000, "2020-01-15"),
    (3, "Charlie", "Marketing", 60000, "2022-01-15"),
    (3, "Charlie", "Marketing", 58000, "2021-01-15"),
    (3, "Charlie", "Marketing", 56000, "2020-01-15"),
]


print(data)

df = spark.createDataFrame(data)

df = df.withColumnRenamed('_1',"id")\
       .withColumnRenamed('_2',"name")\
       .withColumnRenamed('_3',"dept")\
       .withColumnRenamed('_4',"salary")\
       .withColumn("joining_date",col('_5').cast("date"))\
       .withColumn("Year",year(col('joining_date')))\
       .drop('_5')
df

[(1, 'Alice', 'Engineering', 70000, '2022-01-15'), (1, 'Alice', 'Engineering', 68000, '2021-01-15'), (1, 'Alice', 'Engineering', 66000, '2020-01-15'), (2, 'Bob', 'HR', 50000, '2022-01-15'), (2, 'Bob', 'HR', 48000, '2021-01-15'), (2, 'Bob', 'HR', 46000, '2020-01-15'), (3, 'Charlie', 'Marketing', 60000, '2022-01-15'), (3, 'Charlie', 'Marketing', 58000, '2021-01-15'), (3, 'Charlie', 'Marketing', 56000, '2020-01-15')]


DataFrame[id: bigint, name: string, dept: string, salary: bigint, joining_date: date, Year: int]

In [9]:
df_filter = df.select("*")\
              .filter(year(now())-3 <= col('Year'))

In [10]:
windowspec = Window.partitionBy('dept').orderBy("joining_date").rangeBetween(Window.unboundedPreceding,0)

avgsalary = df_filter.withColumn("avg",avg('salary').over(windowspec))

avgsalary

DataFrame[id: bigint, name: string, dept: string, salary: bigint, joining_date: date, Year: int, avg: double]